In [ ]:
import os
import psutil
import time
import math
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import csv

temp = []
missing_values = 0
dataset = 'example_from_paper_0'
path = '/content/drive/MyDrive/ESE 589/'

threshold = 2

In [ ]:
#Node Object Implementation - Attributes: data, count, left node, right node, parent, child - Methods: set/get_data(), set/get_count(),
#set/get_left(), set/get_right(), set/get_parent(), set/get_child()

class Node(object):
    def __init__(self, data=None, count=0, left_node=None, right_node=None, parent=None, depth=None):
        self.data = data
        self.count = count
        self.left_node = left_node
        self.right_node = right_node
        self.parent = parent
        self.depth = depth

    def get_data(self):
        return self.data

    def get_count(self):
        return self.count

    def get_left(self):
        return self.left_node

    def get_right(self):
        return self.right_node

    def get_child(self):
        return self.child

    def get_parent(self):
        return self.parent

    def set_count(self, new_count):
        self.count = new_count

    def set_left(self, new_left):
        self.left_node = new_left

    def set_right(self, new_right):
        self.right_node = new_right
    
    def set_child(self, new_child):
        self.child = new_child

    def set_parent(self, new_parent):
        self.parent = new_parent

In [ ]:
#Tree Class implementation - Attributes: root, head, threshhold, neighbor, parent - Methods: insert_l(), insert_r(), insert_right(), insert_left()
#generate()
class Tree(object):
    COUNT = [10]

    def __init__(self, root=None, threshold=None, neighbor=None, parent=None):
        self.root = root
        self.head = root
        self.threshold = threshold
        self.neighbor = neighbor
        self.parent = parent

    def insert_l(self, data, parent):
        new_node = Node(data=data, count=len(data), parent=self.head)
        self.head.set_left(new_node)
        self.head = new_node

    def insert_r(self, data, parent):
        new_node = Node(data=data, count=len(data), parent=self.head.get_parent)
        self.head.set_right(new_node)
        self.head = new_node

    def insert_left(self, node):
        self.head.set_left(node)
        self.head = node

    def insert_right(self, node):
        self.head.set_right(node)
        self.head = node

    # input: data(star) table
    # generates the star tree based on the datatable by inserting or aggregating each column as a new child
    def generate(self, df):
        num_dim = len(df.columns) - 1
        for i in range(len(df)):
            for j in range(num_dim - 1):
                if self.head.get_left() == None:
                    new_left = Node(data=df.iloc[i, j], count=df.iloc[i, num_dim], parent=self.head, depth=j)
                    self.head.set_left(new_left)
                    self.head = self.head.get_left()
                else:
                    self.head = self.head.get_left()
                    while self.head.get_right() != None and self.head.get_data() != df.iloc[i, j]:
                        self.head = self.head.get_right()
                    if self.head.get_right() == None and self.head.get_data() != df.iloc[i, j]:
                        new_right = Node(data=df.iloc[i, j], count=df.iloc[i, num_dim], parent=self.head.get_parent(),
                                         depth=j)
                        self.head.set_right(new_right)
                        self.head = self.head.get_right()
                    elif self.head.get_data() == df.iloc[i, j]:
                        self.head.set_count(self.head.get_count() + df.iloc[i, num_dim])
            self.head = self.root

In [ ]:
#Star Cube Algorithm Implementation - Attributes: min_sup, result, base_tree, c_tree, reset - Methods: insert_neighbor(), delete_neighbor(), 
#starcubing(), getpath(), getpathprint(), printpath(), print2DUtil()
class StarCube(object):
    def __init__(self, min_sup=None, base_tree=None):
        self.min_sup = min_sup
        self.result = []
        self.base_tree = base_tree
        self.c_tree = base_tree
        self.reset = 0

    def insert_neighbor(self, tree):
        self.c_tree.neighbor = tree
        tree.parent = self.c_tree
        self.c_tree = self.c_tree.neighbor
        # self.c_tree = self.base_tree

    def delete_neighbor(self, tree):
        while self.c_tree is not None:
            if self.c_tree == tree:
                self.c_tree = None
            self.c_tree = self.c_tree.neighbor

    # input: star tree and star tree root
    # implements the star cubing algorithm
    def starcubing(self, t, cnode):
        f = open("../output.txt", "w+")
        cc = None
        separator = ':'

        # while in the base tree, insert or aggregate the current node to each of its neighboring trees
        # additional printing is added for debugging purposes if the node was skipped
        self.c_tree = self.base_tree
        while self.c_tree.neighbor is not None:
            print(
                separator.join([str(self.c_tree.neighbor.root.get_data()), str(self.c_tree.neighbor.root.get_count())]),
                end='\t')
            if self.c_tree.neighbor.head.depth > cnode.depth:
                print("start traversing up. ", end='')
                while self.c_tree.neighbor.head.get_parent() is not None \
                        and self.c_tree.neighbor.head.get_data().split('_')[0] != cnode.get_data().split('_')[0]:
                    self.c_tree.neighbor.head = self.c_tree.neighbor.head.get_parent()
                    print(self.c_tree.neighbor.head.data, end='--')
                print("end traversal.")
                print()
                print("insert right: ", end='')
                # print(separator.join([str(self.c_tree.neighbor.root.get_data()), str(self.c_tree.neighbor.root.get_count())]), end = '\t')
                if self.c_tree.neighbor.head.depth == cnode.depth:
                    if cnode.data == self.c_tree.neighbor.head.data or self.getpath(
                            cnode) == self.c_tree.neighbor.head.data:
                        self.c_tree.neighbor.head.set_count(self.c_tree.neighbor.head.get_count() + cnode.get_count())
                        print("aggregated ", end='')
                        print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
                    else:
                        new_node = Node(cnode.data, cnode.count, parent=self.c_tree.neighbor.head.parent,
                                        depth=cnode.depth)
                        self.c_tree.neighbor.insert_right(new_node)
                        print("inserted ", end='')
                        print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
                else:
                    print("skipped: ", end='')
                    print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
            else:
                print("insert left: ", end='')
                if self.c_tree.neighbor.head.get_left() is not None:
                    temp = self.c_tree.neighbor.head.get_left()
                    while temp is not None and temp.get_data() != cnode.get_data():
                        temp = temp.get_right()
                    if temp is not None and temp.get_data() == cnode.get_data():
                        self.c_tree.neighbor.head = temp
                        self.c_tree.neighbor.head.set_count(self.c_tree.neighbor.head.get_count() + cnode.get_count())
                        print("aggregated ", end='')
                        print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
                    elif temp is not None and temp.get_parent() is not None:
                        new_node = Node(cnode.data, cnode.count, parent=self.c_tree.neighbor.head.parent,
                                        depth=cnode.depth)
                        self.c_tree.neighbor.insert_right(new_node)
                        print("inserted right: ", end='')
                        print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
                    else:
                        new_node = Node(cnode.data, cnode.count, parent=self.c_tree.neighbor.head, depth=cnode.depth)
                        self.c_tree.neighbor.insert_left(new_node)
                        print("inserted ", end='')
                        print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
                elif self.c_tree.neighbor.head.depth == cnode.depth - 1 or cnode.get_parent().data is None:
                    new_node = Node(cnode.data, cnode.count, parent=self.c_tree.neighbor.head, depth=cnode.depth)
                    self.c_tree.neighbor.insert_left(new_node)
                    print("inserted ", end='')
                    print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
                else:
                    print("skipped: ", end='')
                    print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
            self.c_tree = self.c_tree.neighbor
        print()

        # if count of cnode meets threshold, outputs to file
        # if cnode is not a leaf, create a new neighboring tree and append to the last neighbor
        if cnode.get_count() >= self.min_sup:
            if cnode != t.root:
                f.write(separator.join([str(cnode.get_data()), str(cnode.get_count())]))
                f.write(",")
                printtemp = self.getpathprint(cnode)
                f.write(printtemp)
                f.write("\n")
            if cnode.get_left() is None:
                self.printpath(cnode)
            else:
                cc = Node(self.getpath(cnode), cnode.get_count(), depth=cnode.depth)
                tc = Tree(cc)
                self.insert_neighbor(tc)

                print("neighbor inserted: ", end='')
                print(separator.join([str(cnode.get_data()), str(cnode.get_count())]))

        # recursion
        if cnode.get_left() is not None:
            self.starcubing(t, cnode.get_left())
        # delete the child once all children of the child tree have been processed
        if cc is not None:
            print("Erasing child tree:", end=' ')
            print(tc.root.data)
            print()
            tc.parent.neighbor = None
            print("Erasing child node:", end=' ')
            print(cnode.data)
            print()
            cnode.set_left(None)
        # recursion for siblings
        if cnode.get_right() is not None:
            self.starcubing(t, cnode.get_right())

        f.close()

    # gets the path to the current node
    def getpath(self, node):
        temp = node
        result = []
        separator = ''
        while temp is not None:
            result.insert(0, temp.get_data())
            temp = temp.get_parent()
        return separator.join(result)

    # gets the path to the current node for output file with comma delimiter
    def getpathprint(self, node):
        temp = node
        result = []
        separator = ','
        while temp is not None:
            result.insert(0, temp.get_data())
            temp = temp.get_parent()
        return separator.join(result)

    # directly prints the path to the current node
    def printpath(self, node):
        temp = node
        separator = ':'
        while temp is not None:
            print(separator.join([temp.get_data(), str(temp.get_count())]), end="\t")
            temp = temp.get_parent()
        print()

    # prints current node and its children
    def print2DUtil(self, root):
        separator = ':'
        if (root == None):
            return

        print(separator.join([str(root.get_data()), str(root.get_count())]), end='\t')
        if root.get_left() is not None:
            root = root.get_left()
            print(separator.join([str(root.get_data()), str(root.get_count())]), end='\t')
            while root.get_right() is not None:
                root = root.get_right()
                print(separator.join([str(root.get_data()), str(root.get_count())]), end='\t')
        else:
            print("Leaf: ", end='')
            print(separator.join([str(root.get_data()), str(root.get_count())]), end='\t')
        print()

In [ ]:
#Global Methods: convert(), gen_startable(), decode()

def convert(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

def gen_startable(df):
    replaced = 0
    if df.keys()[0] == 0:
        df = df.rename(columns={0: 'A'})
        replaced = 1

    unique = []
    counts = []
    for i in range(len(df.columns)):
        unique = np.unique(df.iloc[:, i])
        counts.append([])
        for j in range(len(unique)):
            count = len(df.iloc[:, i][df.iloc[:, i] == unique[j]])
            counts[i].append(count)
            if threshold > count:
                df.iloc[:, i][df.iloc[:, i] == unique[j]] = -1
                
    df = df.groupby(df.columns.tolist()).size().reset_index().rename(columns={0: 'count'})

    if replaced == 1:
        df = df.rename(columns={'A': 0})
    return df

# turns integer categories into strings with labels
def decode(startable):
    separator = '_'
    separator2 = ''
    for i in range(len(startable)):
        for j in range(len(startable.columns) - 1):
            startable.iloc[i, j] = separator.join(
                [separator2.join([chr(97 + math.floor(j / 26)) * (math.floor(j / 26)), chr(97 + (j % 26))]),
                 str(int(startable.iloc[i, j]))])
    return startable


In [ ]:
#Missing Value Treatment - inserting '-1' values

with open( path +'data/{dataset}.csv'.format(dataset = dataset), 'r') as read:
  with open( path +'data/preprocessed-{dataset}.csv'.format(dataset = dataset), 'w') as out:
    csv_in=csv.reader(read)
    csv_out=csv.writer(out)
    for row in csv_in:
      #print('row:',row)
      for item in range(len(row)):  
        if row[item] != '?' :
          temp.append(row[item])
          #print("item:", row[item])
        else:
          temp.append(-1)
          missing_values += 1
          #print('missing value:',row[item] )  
      csv_out.writerow(temp)
      temp.clear()
print('MS:',missing_values)

MS: 0


In [ ]:
#Main
#Initiating putill for system & process utilization
process = psutil.Process(os.getpid())
#Initiating the wall time starting point
start_time = time.time()
Wall_time_start = time.monotonic()

f = open(path + 'output-{dataset}_{threshold}.txt'.format(dataset = dataset, threshold = threshold), 'w+')
file = path +'data/preprocessed-{dataset}.csv'.format(dataset = dataset)
#print('input file: ', file)
df = pd.read_csv(file)

#Data Pre-processing - Creating Star Tables
startable = gen_startable(df)
startable = decode(startable)
#Data Pre-processing - Creating Star Trees
root = Node(data='root', count=sum(startable.iloc[:, len(startable.columns) - 1]), depth=0)
startree = Tree(root, threshold)
print('star Tree',startree)
startree.generate(startable)

#Star Cubing Algorithm 
starcube = StarCube(threshold, startree)
starcube.starcubing(startree, root)
#Initiating the wall time ending point
Wall_time_end = time.monotonic()
#Output File Title
print('Dataset:', dataset, file=f)
print('Threshold:', threshold, file=f)
#Missing Values Replaced by -1
print('Missing Values Inserted:', missing_values, file=f)

#Retreving utilities inofrmation from the process
print("--- %s seconds ---" % (time.time() - start_time), file =f)
print('Wall Time', convert(Wall_time_end-Wall_time_start), file =f)
print('Memory Resident Set Size',process.memory_info().rss, file =f)
print('Memory Virtual Memory Size',process.memory_info().vms, file =f)
print('Memory Shared',process.memory_info().shared, file =f)
print('Memory Precent:', process.memory_percent(), file =f)
print('CPU Precent:', process.cpu_percent(), file =f)
print('CPU Time:', process.cpu_times(), file =f)
f.close()


star Tree <__main__.Tree object at 0x7f05fbc2ddd0>

neighbor inserted: root:3
root:3	insert left: skipped: a_-1:1

root:3	insert left: inserted b_-1:1

root:3	insert left: inserted c_-1:1

root:3	start traversing up. b_-1--root--end traversal.

insert right: inserted a_1:2

neighbor inserted: a_1:2
root:3	insert left: inserted b_-1:2
roota_1:2	insert left: inserted b_-1:2

neighbor inserted: b_-1:2
root:3	insert left: inserted c_-1:2
roota_1:2	insert left: inserted c_-1:2
roota_1b_-1:2	insert left: inserted c_-1:2

c_-1:2	b_-1:2	a_1:2	root:3	
Erasing child tree: roota_1b_-1

Erasing child node: b_-1

Erasing child tree: roota_1

Erasing child node: a_1

Erasing child tree: root

Erasing child node: root

